# E-commerce

## Общее описание:


Продакт-менеджер Василий попросил вас проанализировать совершенные покупки и ответить на следующие вопросы:

1. Сколько у нас пользователей, которые совершили покупку только один раз? (7 баллов) 
2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)? (10 баллов)
3. По каждому товару определить, в какой день недели товар чаще всего покупается. (7 баллов)
4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть. (8 баллов) 
5. Используя pandas, проведи когортный анализ пользователей. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц. Описание подхода можно найти тут. (15 баллов)
6. Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию. В кластеризации можешь выбрать следующие метрики: R - время от последней покупки пользователя до текущей даты, F - суммарное количество покупок у пользователя за всё время, M - сумма покупок за всё время. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 2 до 5 заказов в неделю, monetary от 1780 до 3560 рублей в неделю. Описание подхода можно найти тут. (23 балла)

Для решения задачи проведи предварительное исследование данных и сформулируй, что должно считаться покупкой. Обосновать свой выбор ты можешь с помощью фактов оплат, статусов заказов и других имеющихся данных.


### Описание данных


**olist_customers_datase.csv** — таблица с уникальными идентификаторами пользователей

- `customer_id` — позаказный идентификатор пользователя
- `customer_unique_id` —  уникальный идентификатор пользователя  (аналог номера паспорта)
- `customer_zip_code_prefix` —  почтовый индекс пользователя
- `customer_city` —  город доставки пользователя
- `customer_state` —  штат доставки пользователя


**olist_orders_dataset.csv** —  таблица заказов
- `order_id` —  уникальный идентификатор заказа (номер чека)
- `customer_id` —  позаказный идентификатор пользователя
- `order_status` —  статус заказа
- `order_purchase_timestamp` —  время создания заказа
- `order_approved_at` —  время подтверждения оплаты заказа
- `order_delivered_carrier_date` —  время передачи заказа в логистическую службу
- `order_delivered_customer_date` —  время доставки заказа
- `order_estimated_delivery_date` —  обещанная дата доставки


**olist_order_items_dataset.csv** —  товарные позиции, входящие в заказы

- `order_id` —  уникальный идентификатор заказа (номер чека)
- `order_item_id` —  идентификатор товара внутри одного заказа
- `product_id` —  ид товара (аналог штрихкода)
- `seller_id` — ид производителя товара
- `shipping_limit_date` —  максимальная дата доставки продавцом для передачи заказа партнеру по логистике
- `price` —  цена за единицу товара
- `freight_value` —  вес товара

— Пример структуры данных можно визуализировать по order_id == 00143d0f86d6fbd9f9b38ab440ac16f5


**Уникальные статусы заказов в таблице olist_orders_dataset**:

- created —  создан
- approved —  подтверждён
- invoiced —  выставлен счёт
- processing —  в процессе сборки заказа
- shipped —  отгружен со склада
- delivered —  доставлен пользователю
- unavailable —  недоступен
- canceled —  отменён

In [1]:
# Импортирую необходимые для работы библиотеки.
import numpy as np
import pandas as pd

In [2]:
customer = pd.read_csv('olist_customers_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
order_items = pd.read_csv('olist_order_items_dataset.csv')

In [3]:
customer.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [4]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [5]:
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


#### Вывожу количество пропусков (NaN) в данных во всех трёх DataFrame.

In [6]:
customer.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [7]:
orders.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [8]:
order_items.isna().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

#### Вывожу уникальные значения данных во всех трёх DataFrame.

In [9]:
customer.nunique()

customer_id                 99441
customer_unique_id          96096
customer_zip_code_prefix    14994
customer_city                4119
customer_state                 27
dtype: int64

In [10]:
orders.nunique()

order_id                         99441
customer_id                      99441
order_status                         8
order_purchase_timestamp         98875
order_approved_at                90733
order_delivered_carrier_date     81018
order_delivered_customer_date    95664
order_estimated_delivery_date      459
dtype: int64

In [11]:
order_items.nunique()

order_id               98666
order_item_id             21
product_id             32951
seller_id               3095
shipping_limit_date    93318
price                   5968
freight_value           6999
dtype: int64

#### Вывожу типы данных в колонках во всех трёх DataFrame.

In [12]:
customer.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [13]:
orders.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [14]:
order_items.dtypes

order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

#### Вывожу описание числовых колонок во всех трёх DataFrame

In [15]:
customer.describe()

,customer_zip_code_prefix
count,99441.000000
mean,35137.474583
std,29797.938996
min,1003.000000
25%,11347.000000
50%,24416.000000
75%,58900.000000
max,99990.000000


In [16]:
orders.describe()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
count,99441,99441,99441,99441,99281,97658,96476,99441
unique,99441,99441,8,98875,90733,81018,95664,459
top,66dea50a8b16d9b4dee7af250b4be1a5,edb027a75a1449115f6b43211ae02a24,delivered,2018-08-02 12:05:26,2018-02-27 04:31:10,2018-05-09 15:48:00,2018-05-08 19:36:48,2017-12-20 00:00:00
freq,1,1,96478,3,9,47,3,522


In [17]:
order_items.describe()

,order_item_id,price,freight_value
count,112650.000000,112650.000000,112650.000000
mean,1.197834,120.653739,19.990320
std,0.705124,183.633928,15.806405
min,1.000000,0.850000,0.000000
25%,1.000000,39.900000,13.080000
50%,1.000000,74.990000,16.260000
75%,1.000000,134.900000,21.150000
max,21.000000,6735.000000,409.680000


### 1. Сколько у нас пользователей, которые совершили покупку только один раз? (7 баллов) 

In [18]:
# Объединеняю таблицу с уникальными идентификаторами пользователей "customer" и таблицу с заказами "orders" по полю "customer_id".
customer_orders = pd.merge(customer[['customer_id', 'customer_unique_id']], orders[['customer_id', 'order_id', 'order_purchase_timestamp']], on='customer_id')

In [19]:
# Группирую по уникальным пользователям и нахожу количества заказов.
customer_order_count = customer_orders.groupby('customer_unique_id').size()

In [20]:
# Оставляю только пользователей с одним заказом.
single_order_customers = customer_order_count[customer_order_count == 1].count()

In [21]:
# Вывожу итоговый результат.
print(f'{single_order_customers} пользователей, совершили покупку только один раз.')

93099 пользователей, совершили покупку только один раз.


### 2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)? (10 баллов)

In [22]:
# Преобразую время создания заказа "order_purchase_timestamp" в формат datetime.
orders.order_purchase_timestamp = pd.to_datetime(orders.order_purchase_timestamp)

In [23]:
# Добавляю столбец с месяцем заказа.
orders['order_month'] = orders['order_purchase_timestamp'].dt.to_period('M')

In [24]:
# Нахожу недоставленные заказы.
undelivered_orders = orders[orders['order_status'].isin(['canceled', 'unavailable'])]

In [25]:
# Строю таблицу, показывающую, сколько недоставленных заказов было в каждом месяце по каждой из причин.
undelivered_by_month = undelivered_orders.groupby(['order_month', 'order_status']).size().unstack().fillna(0)

In [26]:
# Считаю среднее число.
average_undelivered = undelivered_by_month.mean()

In [27]:
print('Таблица, показывающая какое колличество заказов в месяц в среднем не доставляется по разным причинам:')
average_undelivered

Таблица, показывающая какое колличество заказов в месяц в среднем не доставляется по разным причинам:


order_status
canceled       26.041667
unavailable    25.375000
dtype: float64

### 3. По каждому товару определить, в какой день недели товар чаще всего покупается. (7 баллов)

In [28]:
# Добавляю столбец с названием дня недели.
orders['order_weekday'] = orders['order_purchase_timestamp'].dt.day_name()

In [29]:
# Объединяю таблицу с товарными позициями, входящими в заказы "order_items" и таблицу с заказами "orders" по полю "order_id".
order_items_with_orders = pd.merge(order_items, orders[['order_id', 'order_weekday']], on='order_id')

In [30]:
# С помощью lambda-функции в агрегации нахожу дни недели, когда товар покупается чаще всего.
most_common_weekday = order_items_with_orders.groupby('product_id')['order_weekday'].agg(lambda x: x.mode()[0]).reset_index()

In [31]:
print(f'Таблица с колонками id товара "product_id" и днём, когда его покупают чаще всего "order_weekday":')
most_common_weekday

Таблица с колонками id товара "product_id" и днём, когда его покупают чаще всего "order_weekday":


,product_id,order_weekday
0,00066f42aeeb9f3007548bb9d3f33c38,Sunday
1,00088930e925c41fd95ebfe695fd2655,Tuesday
2,0009406fd7479715e4bef61dd91f2462,Thursday
3,000b8f95fcb9e0096488278317764d19,Friday
4,000d9be29b5207b54e86aa1b1ac54872,Tuesday
...,...,...
32946,fff6177642830a9a94a0f2cba5e476d1,Saturday
32947,fff81cc3158d2725c0655ab9ba0f712c,Monday
32948,fff9553ac224cec9d15d49f5a263411f,Friday
32949,fffdb2d0ec8d6a61f0a0a0db3f25b441,Tuesday


### 4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть. (8 баллов) 

In [32]:
# Преобразую столбец "order_purchase_timestamp" в тип datetime.
customer_orders.order_purchase_timestamp = pd.to_datetime(customer_orders.order_purchase_timestamp)

In [33]:
# Добавляю столбец с месяцем заказа.
customer_orders['order_month'] = customer_orders.order_purchase_timestamp.dt.to_period('M')

In [34]:
# Добавляю колонку с количеством дней в соответствующим месяце.
customer_orders['days_in_month'] = customer_orders['order_purchase_timestamp'].dt.days_in_month

In [35]:
# Добавляю колонку с колличеством неполных недель в соответствующим месяце.
customer_orders['weeks_in_month'] = customer_orders['days_in_month'] / 7

In [36]:
# Нахожу среднее количество заказов на одного пользователя в неделю с учётом числа  недель в соответствующий месяц.
avg_orders_per_week = customer_orders.groupby(['customer_unique_id', 'order_month'], as_index=False) \
.agg({'order_purchase_timestamp':'count', 'weeks_in_month':'mean'})

In [37]:
# Считаю среднее количество покупок в неделю.
avg_orders_per_week['average_purchases_per_week'] = \
avg_orders_per_week['order_purchase_timestamp'] / avg_orders_per_week['weeks_in_month']

In [38]:
print(f'Таблица показывающая сколько у каждого из пользователей в среднем покупок в неделю (по месяцам):')
avg_orders_per_week

Таблица показывающая сколько у каждого из пользователей в среднем покупок в неделю (по месяцам):


,customer_unique_id,order_month,order_purchase_timestamp,weeks_in_month,average_purchases_per_week
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,1,4.428571,0.225806
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05,1,4.428571,0.225806
2,0000f46a3911fa3c0805444483337064,2017-03,1,4.428571,0.225806
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10,1,4.428571,0.225806
4,0004aac84e0df4da2b147fca70cf8255,2017-11,1,4.285714,0.233333
...,...,...,...,...,...
98041,fffcf5a5ff07b0908bd4e2dbc735a684,2017-06,1,4.285714,0.233333
98042,fffea47cd6d3cc0a88bd621562a9d061,2017-12,1,4.428571,0.225806
98043,ffff371b4d645b6ecea244b27531430a,2017-02,1,4.000000,0.250000
98044,ffff5962728ec6157033ef9805bacc48,2018-05,1,4.428571,0.225806


### 5. Используя pandas, проведи когортный анализ пользователей. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц. Описание подхода можно найти тут. (15 баллов)

In [39]:
# Находим дату первой покупки для каждого пользователя.
first_order_dates = customer_orders.groupby('customer_unique_id')['order_purchase_timestamp'].min()

In [40]:
# Создаю колонку с месяцем первой покупки.
customer_orders['first_order_month'] = customer_orders['customer_unique_id'].map(first_order_dates).dt.to_period('M')

In [41]:
# Группирую по месяцу первой покупки и месяцам заказа, и считаю уникальных пользователей.
cohort_data = customer_orders.groupby(['first_order_month', 'order_month'])['customer_unique_id'].nunique().unstack()

In [42]:
# Считаю количество пользователей на 1-ый и 3-ий месяц.
first_month_users = cohort_data.iloc[:, 0]
third_month_users = cohort_data.iloc[:, 2]
# (При использовании "shift" программа рекомендует использовать "iloc")

In [43]:
# Считаю retention на 3-ий месяц как отношение количества пользователей на 3-й месяц к их числу в первый месяц.
retention_3_month = third_month_users / first_month_users

In [44]:
# Нахожу когорту с самым высоким retention.
max_retention_3_month = retention_3_month.sort_values(ascending=False).head(1)

In [45]:
print('Когорта с самым высоким retention на 3й месяц:')
max_retention_3_month

Когорта с самым высоким retention на 3й месяц:


first_order_month
2016-09   NaN
Freq: M, dtype: float64

### 6. Построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию.  В кластеризации можешь выбрать следующие метрики: R - время от последней покупки пользователя до текущей даты, F - суммарное количество покупок у пользователя за всё время, M - сумма покупок за всё время. Подробно опиши, как ты создавал кластеры.       Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров.                                                        Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 2 до 5 заказов в неделю, monetary от 1780 до 3560 рублей в неделю. Описание подхода можно найти тут. (23 балла)

In [46]:
# Объединеняю таблицы "customer_orders" и "price" по полю "order_id".
customer_orders = pd.merge(customer_orders, order_items[['order_id', 'price']], on='order_id')

In [47]:
# Нахожу текущаю дату (дату последнего оформления заказа).
current_date = orders.order_purchase_timestamp.max()
current_date

Timestamp('2018-10-17 17:30:18')

In [48]:
# Расчитываю RFM 
# (R - recency (время от последней покупки пользователя до текущей даты),
# F - frequency (суммарное количество покупок у пользователя за всё время),
# M - monetary (сумма покупок за всё время))
rfm = customer_orders.groupby('customer_unique_id').agg({
    'order_purchase_timestamp': lambda x: (current_date - x.max()).days,
    'order_id': 'count',
    'price': 'sum'
}).reset_index()

In [49]:
# Переименовываю название столбцов.
rfm.columns = ['customer_unique_id', 'Recency', 'Frequency', 'Monetary']

In [50]:
# Разделяю на квантили для сегментации.
r_quantiles = rfm['Recency'].quantile([0.25, 0.5, 0.75])
f_quantiles = rfm['Frequency'].quantile([0.25, 0.5, 0.75])
m_quantiles = rfm['Monetary'].quantile([0.25, 0.5, 0.75])

In [51]:
# Функции, осуществляющие сегментацию.
def r_score(x):
    if x <= r_quantiles[0.25]:
        return 1
    elif x <= r_quantiles[0.5]:
        return 2
    elif x <= r_quantiles[0.75]:
        return 3
    else:
        return 4

def f_score(x):
    if x <= f_quantiles[0.25]:
        return 4
    elif x <= f_quantiles[0.5]:
        return 3
    elif x <= f_quantiles[0.75]:
        return 2
    else:
        return 1

def m_score(x):
    if x <= m_quantiles[0.25]:
        return 4
    elif x <= m_quantiles[0.5]:
        return 3
    elif x <= m_quantiles[0.75]:
        return 2
    else:
        return 1

In [52]:
# Примененяю apply-функцию для сегментации и добавления соответствующих колонок в таблицу.
rfm['R'] = rfm['Recency'].apply(r_score)
rfm['F'] = rfm['Frequency'].apply(f_score)
rfm['M'] = rfm['Monetary'].apply(m_score)

In [53]:
# Строю RFM сегменты, объединяя их 'в слово'.
rfm['RFM_Segment'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)

In [54]:
print('Итоговая таблица с указанием сегментации пользователей')
rfm

Итоговая таблица с указанием сегментации пользователей


,customer_unique_id,Recency,Frequency,Monetary,R,F,M,RFM_Segment
0,0000366f3b9a7992bf8c76cfdf3221e2,160,1,129.90,1,4,2,142
1,0000b849f77a49e4a4ce2b2a4ca5be3f,163,1,18.90,1,4,4,144
2,0000f46a3911fa3c0805444483337064,585,1,69.00,4,4,3,443
3,0000f6ccb0745a6a4b88665a16c9f078,369,1,25.99,3,4,4,344
4,0004aac84e0df4da2b147fca70cf8255,336,1,180.00,3,4,1,341
...,...,...,...,...,...,...,...,...
95415,fffcf5a5ff07b0908bd4e2dbc735a684,495,2,1570.00,4,1,1,411
95416,fffea47cd6d3cc0a88bd621562a9d061,310,1,64.89,3,4,3,343
95417,ffff371b4d645b6ecea244b27531430a,617,1,89.90,4,4,3,443
95418,ffff5962728ec6157033ef9805bacc48,168,1,115.00,2,4,2,242
